In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

# Imports and feature selection

In [231]:
# using docs to model
# labels come from claims data 
claims = pd.read_csv('/Users/drewrice/Desktop/capson_csv/Claims.csv')
docs = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ProviderInfo.csv')
prems = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfWrittenPremium.csv')
debs_and_creds = pd.read_csv('/Users/drewrice/Desktop/capson_csv/ScheduleOfDebitsAndCredits.csv')

In [232]:
# drop
docs.drop(['Middle Name','Address 1', 'Address 2', 'Zip','Address Type','Practice Start Date'],axis=1,inplace=True)
docs.drop_duplicates(inplace=True)

In [233]:
indem = claims[['Indemnity_Paid','ProviderId']]

In [234]:
# docs DF, conversion to string
# lower ProviderId
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']] = \
docs[['ProviderId','Last Name','First Name','Gender','City','State','County']].astype(str)
docs['ProviderId'] = docs['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())

In [235]:
# indem DF, conversions to string and float
# lower ProviderId
indem['Indemnity_Paid'] = indem['Indemnity_Paid'].str.replace(r'[$,]', '').astype('float')
indem['ProviderId'] = indem['ProviderId'].astype(str)
indem['ProviderId'] = indem['ProviderId'].str.lower()

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.h

In [236]:
def build_classifier_label(item):
    if item == 0:
        return 'suit, no indem'
    if item != 0:
        return 'suit, indem'

In [237]:
indem['indem_label'] = indem['Indemnity_Paid'].apply(lambda item: build_classifier_label(item))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


# Join docs and indemnity tables, fill NaNs with 'no suit'

In [295]:
joined = pd.merge(docs, indem[['ProviderId', 'indem_label']], how='outer', on='ProviderId')
joined.drop_duplicates(inplace=True)

In [296]:
# finish indemnity classifier
# find NaNs in indem_label column, replace with 'no suit'
def indem_label_no_suit(x):
    if type(x) == float: 
        if np.isnan(x): 
            return 'no suit'
    else:
        return x

In [297]:
joined.indem_label = joined.indem_label.apply(lambda item: indem_label_no_suit(item))

# Claims data
#### building classifiers for BoardCredit and ClaimsBand, then merging with joined

In [298]:
def build_classifier(item):
    if item == 0:
        return 0
    if item != 0:
        return 1

In [299]:
# resave debs_and_creds with just the desired columns
debs_and_creds = debs_and_creds[['ProviderId','BoardCredit','ClaimsBand']]

# lower Provider Id
# reset index required
debs_and_creds['ProviderId'] = debs_and_creds['ProviderId'].map(lambda x: x if type(x)!=str else x.lower())
debs_and_creds.tail()

,ProviderId,BoardCredit,ClaimsBand
3297,ffa90045-9407-48c9-911b-6abb34e91474,-81,-242
3298,ffae8b53-a351-4912-8318-5ee098fb41bc,-1290,-4902
3299,ffc00eee-65ac-4691-87a1-894c80dc0c05,-2768,-13832
3300,ffdb6106-28fc-459e-997b-1c85900396a6,0,-9948
3301,fff2ce54-e2a9-42fd-96e1-516fd11977c7,-1190,-5948


In [300]:
# group by Provider ID, resave debs_and_creds
debs_and_creds = debs_and_creds.groupby(debs_and_creds['ProviderId']).sum()
debs_and_creds.reset_index(inplace=True)

In [301]:
# use build_classifier to create binary column from BoardCredit & ClaimsBand
debs_and_creds['board_cert'] = debs_and_creds['BoardCredit'].apply(lambda item: build_classifier(item))
debs_and_creds['claims_band'] = debs_and_creds['ClaimsBand'].apply(lambda item: build_classifier(item))

In [302]:
joined = pd.merge(joined, debs_and_creds[['ProviderId','board_cert', 'claims_band']], how='outer', on='ProviderId')

# EDA

In [303]:
print 'Doctors wih no suits: ', joined[joined['indem_label'] == 'no suit'].sort(['Last Name']).shape[0]
print 'Doctors with suits, no indemnity paid: ', joined[joined['indem_label'] == 'suit, no indem'].sort(['Last Name']).shape[0]
print 'Doctors with suits, indemnity paid: ', joined[joined['indem_label'] == 'suit, indem'].sort(['Last Name']).shape[0]

Doctors wih no suits:  6147
Doctors with suits, no indemnity paid:  282
Doctors with suits, indemnity paid:  43


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  app.launch_new_instance()


# Feature engineering

### ~ build age column ~

In [304]:
# convert DoB to datetime object
joined['Date of Birth'] = pd.to_datetime(joined['Date of Birth'])

In [305]:
# time_delta function
def time_delta(dt):
    if dt.year > 2000:
        dt = dt - timedelta(days=36525)
    return dt

In [306]:
# apply time_delta function to DoB column, deal with datetime rounding to the 2000s
joined['Date of Birth'] = joined['Date of Birth'].apply(lambda dt: time_delta(dt))

In [307]:
# fill missing values with 1/1/2060
joined['Date of Birth'].fillna(datetime(2060, 1, 1, 0, 0),inplace=True)

In [308]:
# build age column
now = datetime.now()
joined['age'] = now - joined['Date of Birth']

In [309]:
def to_years(dt):
    if dt:
        dt = (dt.days) / 365.25
    return dt

In [310]:
# TODO: convert days to years in joined['age']
# TODO: drop 1/1/2060 instances
joined['age'] = (joined['age'].apply(lambda dt: to_years(dt))).round()

In [315]:
# refill Date of Birth with NaNs
joined['Date of Birth'].replace(datetime(2060, 1, 1, 0, 0), np.nan, inplace=True)

In [317]:
def refill_age(x):
    if x < 0:
        x = np.nan
    return x

In [318]:
joined['age'] = (joined['age'].apply(lambda dt: refill_age(dt))).round()

In [319]:
# joined

,ProviderId,Last Name,First Name,Gender,Date of Birth,City,State,County,indem_label,board_cert,claims_band,age
0,2b08b6ae-6427-4531-bf5f-51c90685f1df,1,Slot,Unknown,NaT,Auburn,AL,LEE,no suit,NaN,NaN,NaN
1,9977d6b0-04eb-4310-be27-8470257da33e,Abalos,Jennifer,Unknown,1972-10-01,Plano,TX,DALLAS,no suit,0.0,0.0,44.0
2,5ed9f93e-5bf2-4990-bd31-fb0754ae3aec,Abbasi,Noma,Unknown,1971-11-01,Sugar Land,TX,FORT BEND,no suit,1.0,1.0,45.0
3,949a1121-3f0f-41f2-a6df-b4acc1b64f9a,Abben,Richard,Male,1952-10-11,Houma,LA,TERREBONNE,no suit,1.0,1.0,64.0
4,6422a3d2-1405-4511-804f-12733e29c412,Abbott,Abraham,Unknown,NaT,nan,nan,nan,no suit,NaN,NaN,NaN
5,4c67e308-df98-42ff-b424-06516486d2b4,Abburi,Madhava,Male,1971-02-26,nan,nan,nan,no suit,1.0,1.0,45.0
6,e35196bd-a7a2-4f07-95e9-4a47775e0ac8,Abdul Rashid,Nik,Unknown,1970-03-30,Las Vegas,NV,CLARK,no suit,1.0,1.0,46.0
7,f9f98580-e6db-4428-8fab-6563b8574c22,Abdulla,Amina,Female,1946-07-30,Houston,TX,HARRIS,no suit,0.0,1.0,70.0
8,4590a5c6-a9d8-4b5e-946f-955f7373eade,Aboubaker,Kaiss,Unknown,1973-08-04,Dallas,TX,DALLAS,no suit,0.0,0.0,43.0
9,805a9da9-be65-45fc-9e7e-77cbe4a37036,Aboyeji,Olufemi,Male,1978-09-26,Schererville,IN,LAKE,no suit,1.0,1.0,38.0
